In [ ]:

import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import Adam, LBFGS

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

class RLC_PINN(nn.Module):
    """
    Physics-Informed Neural Network for RLC Circuit
    """
    def __init__(self, layers, R=10.0, L=0.1, C=0.01):
        super(RLC_PINN, self).__init__()
        
        # Circuit parameters
        self.R = R  # Resistance (Ohms)
        self.L = L  # Inductance (Henry)
        self.C = C  # Capacitance (Farad)
        
        # Network layers
        self.linears = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.linears.append(nn.Linear(layers[i], layers[i+1]))
        
        # Initialize weights
        self.init_weights()
        
        # Activation function
        self.activation = torch.tanh
    
    def init_weights(self):
        """Initialize network weights using Xavier initialization"""
        for m in self.linears:
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, t):
        """
        Forward pass: t -> Q(t)
        Input: time t
        Output: charge Q(t)
        """
        x = t
        for i, layer in enumerate(self.linears[:-1]):
            x = self.activation(layer(x))
        x = self.linears[-1](x)  # No activation on output
        return x
    
    def voltage_source(self, t):
        """
        Voltage source function: V(t) = 5V step function
        """
        return 5.0 * torch.ones_like(t)  # 5V DC step
    
    def compute_current(self, t):
        """
        Compute current I(t) = dQ/dt using automatic differentiation
        """
        t.requires_grad_(True)
        Q = self.forward(t)
        
        I = autograd.grad(Q, t,
                         grad_outputs=torch.ones_like(Q),
                         create_graph=True)[0]
        return I
    
    def physics_loss(self, t_physics):
        """
        Compute physics loss based on RLC circuit equations
        L * d²Q/dt² + R * dQ/dt + Q/C = V_source(t)
        """
        t_physics.requires_grad_(True)
        Q = self.forward(t_physics)
        
        # First derivative: dQ/dt = I(t)
        Q_t = autograd.grad(Q, t_physics,
                           grad_outputs=torch.ones_like(Q),
                           create_graph=True)[0]
        
        # Second derivative: d²Q/dt² = dI/dt
        Q_tt = autograd.grad(Q_t, t_physics,
                            grad_outputs=torch.ones_like(Q_t),
                            create_graph=True)[0]
        
        # Voltage source
        V_source = self.voltage_source(t_physics)
        
        # RLC equation residual
        # L * d²Q/dt² + R * dQ/dt + Q/C - V_source(t) = 0
        residual = self.L * Q_tt + self.R * Q_t + Q / self.C - V_source
        
        loss_physics = torch.mean(residual**2)
        return loss_physics
    
    def initial_conditions_loss(self, t_initial):
        """
        Enforce initial conditions:
        Q(0) = 0 (no initial charge)
        I(0) = dQ/dt(0) = 0 (no initial current)
        """
        t_initial.requires_grad_(True)
        Q_initial = self.forward(t_initial)
        
        # dQ/dt at t=0
        I_initial = autograd.grad(Q_initial, t_initial,
                                 grad_outputs=torch.ones_like(Q_initial),
                                 create_graph=True)[0]
        
        # Initial condition losses
        loss_Q_initial = torch.mean(Q_initial**2)  # Q(0) = 0
        loss_I_initial = torch.mean(I_initial**2)  # I(0) = 0
        
        return loss_Q_initial + loss_I_initial
    
    def total_loss(self, t_physics, t_initial, t_data=None, Q_data=None):
        """
        Compute total loss function
        """
        # Physics loss
        loss_physics = self.physics_loss(t_physics)
        
        # Initial conditions loss
        loss_initial = self.initial_conditions_loss(t_initial)
        
        # Data loss (if available)
        loss_data = torch.tensor(0.0)
        if t_data is not None and Q_data is not None:
            Q_pred = self.forward(t_data)
            loss_data = torch.mean((Q_pred - Q_data)**2)
        
        # Weighted total loss
        lambda_physics = 1.0
        lambda_initial = 100.0  # Higher weight for initial conditions
        lambda_data = 10.0
        
        total = (lambda_physics * loss_physics + 
                lambda_initial * loss_initial + 
                lambda_data * loss_data)
        
        return total, loss_physics, loss_initial, loss_data

def analytical_solution(t, R=10.0, L=0.1, C=0.01, V0=5.0):
    """
    Analytical solution for RLC circuit with step input
    """
    # Calculate circuit parameters
    omega_0 = 1.0 / np.sqrt(L * C)  # Natural frequency
    alpha = R / (2 * L)             # Damping coefficient
    
    # Discriminant for determining solution type
    discriminant = alpha**2 - omega_0**2
    
    if discriminant > 0:
        # Overdamped case
        s1 = -alpha + np.sqrt(discriminant)
        s2 = -alpha - np.sqrt(discriminant)
        
        A = -V0 * C * s2 / (s1 - s2)
        B = V0 * C * s1 / (s1 - s2)
        
        Q = V0 * C + A * np.exp(s1 * t) + B * np.exp(s2 * t)
        
    elif discriminant < 0:
        # Underdamped case (most common)
        omega_d = np.sqrt(omega_0**2 - alpha**2)  # Damped frequency
        
        Q = V0 * C * (1 - np.exp(-alpha * t) * 
                      (np.cos(omega_d * t) + (alpha / omega_d) * np.sin(omega_d * t)))
        
    else:
        # Critically damped case
        Q = V0 * C * (1 - (1 + alpha * t) * np.exp(-alpha * t))
    
    return Q

def train_rlc_pinn():
    """
    Train the RLC circuit PINN
    """
    print("="*60)
    print("RLC CIRCUIT MODELING WITH PHYSICS-INFORMED NEURAL NETWORKS")
    print("="*60)
    print()
    print("Circuit Parameters:")
    print("- Resistance (R): 10.0 Ω")
    print("- Inductance (L): 0.1 H") 
    print("- Capacitance (C): 0.01 F")
    print("- Voltage Source: 5V step function")
    print()
    
    # Circuit parameters
    R, L, C = 10.0, 0.1, 0.01
    
    # Time domain
    t_min, t_max = 0.0, 1.0
    
    # Training points
    n_physics = 1000
    n_initial = 10
    
    # Physics collocation points
    t_physics = torch.linspace(t_min, t_max, n_physics).reshape(-1, 1)
    t_physics.requires_grad_(True)
    
    # Initial condition points
    t_initial = torch.zeros(n_initial, 1)
    
    # Initialize network
    layers = [1, 50, 50, 50, 1]  # 1 input (time), 3 hidden layers, 1 output (charge)
    model = RLC_PINN(layers, R=R, L=L, C=C)
    
    # Optimizer
    optimizer = Adam(model.parameters(), lr=0.001)
    
    # Training parameters
    epochs = 10000
    print_interval = 2000
    
    # Training loop
    print("Starting training...")
    print()
    
    loss_history = []
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Compute losses
        total_loss, loss_physics, loss_initial, loss_data = model.total_loss(
            t_physics, t_initial
        )
        
        # Backpropagation
        total_loss.backward()
        optimizer.step()
        
        loss_history.append(total_loss.item())
        
        # Print progress
        if epoch % print_interval == 0:
            print(f"Epoch {epoch:5d} | "
                  f"Total Loss: {total_loss.item():.6f} | "
                  f"Physics: {loss_physics.item():.6f} | "
                  f"Initial: {loss_initial.item():.6f}")
    
    print()
    print("Training completed!")
    print()
    
    return model, loss_history

def fine_tune_with_lbfgs(model, t_physics, t_initial):
    """
    Fine-tune the model using L-BFGS optimizer
    """
    print("Fine-tuning with L-BFGS optimizer...")
    
    # L-BFGS optimizer
    optimizer_lbfgs = LBFGS(model.parameters(), 
                           lr=1.0, 
                           max_iter=1000,
                           max_eval=1000,
                           tolerance_grad=1e-7,
                           tolerance_change=1.0*np.finfo(float).eps,
                           history_size=50,
                           line_search_fn="strong_wolfe")
    
    def closure():
        optimizer_lbfgs.zero_grad()
        total_loss, _, _, _ = model.total_loss(t_physics, t_initial)
        total_loss.backward()
        return total_loss
    
    optimizer_lbfgs.step(closure)
    
    # Final loss
    with torch.no_grad():
        final_loss, _, _, _ = model.total_loss(t_physics, t_initial)
        print(f"Final loss after L-BFGS: {final_loss.item():.8f}")
    
    print()

def analyze_results(model):
    """
    Analyze and visualize the results
    """
    print("Analyzing results...")
    
    # Test time points
    t_test = torch.linspace(0, 1, 200).reshape(-1, 1)
    
    with torch.no_grad():
        # PINN predictions
        Q_pred = model.forward(t_test)
        I_pred = model.compute_current(t_test)
        
        # Convert to numpy for plotting
        t_np = t_test.numpy().flatten()
        Q_pred_np = Q_pred.numpy().flatten()
        I_pred_np = I_pred.numpy().flatten()
        
        # Analytical solution
        Q_analytical = analytical_solution(t_np, R=model.R, L=model.L, C=model.C)
        I_analytical = np.gradient(Q_analytical, t_np)  # dQ/dt
        
        # Voltage across components
        V_R = model.R * I_pred_np
        V_L = model.L * np.gradient(I_pred_np, t_np)
        V_C = Q_pred_np / model.C
        V_source = 5.0 * np.ones_like(t_np)
    
    # Calculate errors
    mse_Q = np.mean((Q_pred_np - Q_analytical)**2)
    mse_I = np.mean((I_pred_np - I_analytical)**2)
    
    print(f"Mean Squared Error (Charge): {mse_Q:.8f}")
    print(f"Mean Squared Error (Current): {mse_I:.8f}")
    print()
    
    # Create comprehensive plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('RLC Circuit Analysis with PINNs', fontsize=16, fontweight='bold')
    
    # Plot 1: Charge vs Time
    axes[0, 0].plot(t_np, Q_analytical, 'r-', linewidth=2, label='Analytical')
    axes[0, 0].plot(t_np, Q_pred_np, 'b--', linewidth=2, label='PINN')
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Charge Q(t) (C)')
    axes[0, 0].set_title('Capacitor Charge')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Current vs Time
    axes[0, 1].plot(t_np, I_analytical, 'r-', linewidth=2, label='Analytical')
    axes[0, 1].plot(t_np, I_pred_np, 'b--', linewidth=2, label='PINN')
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Current I(t) (A)')
    axes[0, 1].set_title('Circuit Current')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Error Analysis
    error_Q = np.abs(Q_pred_np - Q_analytical)
    error_I = np.abs(I_pred_np - I_analytical)
    axes[0, 2].plot(t_np, error_Q, 'g-', linewidth=2, label='Charge Error')
    axes[0, 2].plot(t_np, error_I, 'm-', linewidth=2, label='Current Error')
    axes[0, 2].set_xlabel('Time (s)')
    axes[0, 2].set_ylabel('Absolute Error')
    axes[0, 2].set_title('Prediction Errors')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    axes[0, 2].set_yscale('log')
    
    # Plot 4: Component Voltages
    axes[1, 0].plot(t_np, V_source, 'k-', linewidth=2, label='Source V(t)')
    axes[1, 0].plot(t_np, V_R, 'r-', linewidth=2, label='Resistor V_R')
    axes[1, 0].plot(t_np, V_L, 'g-', linewidth=2, label='Inductor V_L')
    axes[1, 0].plot(t_np, V_C, 'b-', linewidth=2, label='Capacitor V_C')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Voltage (V)')
    axes[1, 0].set_title('Component Voltages')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 5: Kirchhoff's Voltage Law Check
    V_sum = V_R + V_L + V_C
    axes[1, 1].plot(t_np, V_source, 'k-', linewidth=2, label='V_source')
    axes[1, 1].plot(t_np, V_sum, 'r--', linewidth=2, label='V_R + V_L + V_C')
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Voltage (V)')
    axes[1, 1].set_title("Kirchhoff's Voltage Law Verification")
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Plot 6: Phase Portrait (Current vs Charge)
    axes[1, 2].plot(Q_analytical, I_analytical, 'r-', linewidth=2, label='Analytical')
    axes[1, 2].plot(Q_pred_np, I_pred_np, 'b--', linewidth=2, label='PINN')
    axes[1, 2].set_xlabel('Charge Q(t) (C)')
    axes[1, 2].set_ylabel('Current I(t) (A)')
    axes[1, 2].set_title('Phase Portrait (I vs Q)')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print circuit analysis
    print("Circuit Analysis:")
    print(f"- Natural frequency ω₀: {1/np.sqrt(model.L * model.C):.2f} rad/s")
    print(f"- Damping coefficient α: {model.R/(2*model.L):.2f} s⁻¹")
    
    discriminant = (model.R/(2*model.L))**2 - (1/(model.L*model.C))
    if discriminant > 0:
        print("- Circuit type: Overdamped")
    elif discriminant < 0:
        print("- Circuit type: Underdamped")
        omega_d = np.sqrt(1/(model.L*model.C) - (model.R/(2*model.L))**2)
        print(f"- Damped frequency ωd: {omega_d:.2f} rad/s")
    else:
        print("- Circuit type: Critically damped")
    
    print(f"- Final steady-state charge: {Q_pred_np[-1]:.4f} C")
    print(f"- Theoretical steady-state: {5.0 * model.C:.4f} C")

def demonstrate_parameter_discovery():
    """
    Demonstrate inverse problem: discovering circuit parameters from data
    """
    print("\n" + "="*60)
    print("INVERSE PROBLEM: PARAMETER DISCOVERY")
    print("="*60)
    
    # Generate synthetic "experimental" data
    t_data = torch.linspace(0, 1, 50).reshape(-1, 1)
    R_true, L_true, C_true = 15.0, 0.05, 0.02  # Unknown parameters
    
    # Generate noisy data
    with torch.no_grad():
        Q_true = torch.tensor(analytical_solution(t_data.numpy().flatten(), 
                                                 R=R_true, L=L_true, C=C_true), 
                             dtype=torch.float32).reshape(-1, 1)
        # Add noise
        noise_level = 0.02
        Q_data = Q_true + noise_level * torch.randn_like(Q_true)
    
    class InverseRLC_PINN(RLC_PINN):
        """Extended PINN that learns circuit parameters"""
        def __init__(self, layers):
            super().__init__(layers, R=1.0, L=1.0, C=1.0)  # Initial guess
            
            # Make parameters learnable
            self.R = nn.Parameter(torch.tensor(10.0))  # Initial guess
            self.L = nn.Parameter(torch.tensor(0.1))   # Initial guess  
            self.C = nn.Parameter(torch.tensor(0.01))  # Initial guess
    
    # Initialize inverse model
    inverse_model = InverseRLC_PINN([1, 50, 50, 50, 1])
    optimizer = Adam(inverse_model.parameters(), lr=0.001)
    
    # Training points
    t_physics = torch.linspace(0, 1, 500).reshape(-1, 1)
    t_initial = torch.zeros(10, 1)
    
    print("Training inverse model to discover parameters...")
    print(f"True parameters: R={R_true}Ω, L={L_true}H, C={C_true}F")
    print()
    
    # Training loop
    for epoch in range(5000):
        optimizer.zero_grad()
        
        total_loss, _, _, _ = inverse_model.total_loss(
            t_physics, t_initial, t_data, Q_data
        )
        
        total_loss.backward()
        optimizer.step()
        
        if epoch % 1000 == 0:
            print(f"Epoch {epoch:4d} | Loss: {total_loss.item():.6f} | "
                  f"R: {inverse_model.R.item():.2f} | "
                  f"L: {inverse_model.L.item():.4f} | " 
                  f"C: {inverse_model.C.item():.4f}")
    
    print()
    print("Parameter Discovery Results:")
    print(f"Discovered R: {inverse_model.R.item():.2f} Ω (True: {R_true} Ω)")
    print(f"Discovered L: {inverse_model.L.item():.4f} H (True: {L_true} H)")
    print(f"Discovered C: {inverse_model.C.item():.4f} F (True: {C_true} F)")
    
    # Calculate errors
    R_error = abs(inverse_model.R.item() - R_true) / R_true * 100
    L_error = abs(inverse_model.L.item() - L_true) / L_true * 100  
    C_error = abs(inverse_model.C.item() - C_true) / C_true * 100
    
    print(f"Parameter Errors: R: {R_error:.1f}%, L: {L_error:.1f}%, C: {C_error:.1f}%")

def main():
    """
    Main function to run the RLC circuit PINN example
    """
    print("RLC Circuit PINN Example")
    print("Series RLC Circuit Diagram:")
    print("""
    Correct Series RLC Circuit:
    
    +──────[R=10Ω]──────[L=0.1H]──────[C=0.01F]──────+
    |                                                |
    |                                                |
    +                                                |
   V(t)                                              |
   5V DC                                             |
    -                                                |
    |                                                |
    +────────────────────────────────────────────────+
                         GND

    Linear representation:
    
    V_source ──[R=10Ω]──[L=0.1H]──[C=0.01F]── GND
       5V        ↓         ↓         ↓
              V_R=I×R   V_L=L×dI/dt  V_C=Q/C

    Current flows in ONE PATH: I(t) →→→→→→→→→→→→→→→→→
    
    Components connected in SERIES (no branches):
    • Same current I(t) flows through all components
    • Voltages add up: V_source = V_R + V_L + V_C
    """)
    
    # Train the forward model
    model, loss_history = train_rlc_pinn()
    
    # Fine-tune with L-BFGS
    t_physics = torch.linspace(0, 1, 1000).reshape(-1, 1)
    t_initial = torch.zeros(10, 1)
    fine_tune_with_lbfgs(model, t_physics, t_initial)
    
    # Analyze results
    analyze_results(model)
    
    # Demonstrate parameter discovery
    demonstrate_parameter_discovery()
    
    print("\n" + "="*60)
    print("ANALYSIS COMPLETE!")
    print("="*60)

if __name__ == "__main__":
    main()